In [10]:
import numpy as np
import cv2
import tkinter as tk
from tkinter import *
from tkinter import filedialog
import os
import PIL
from PIL import Image, ImageTk
import PySimpleGUI as sg


#########################################
# Best Configuration by file:           #
# eye01.jpg: T87, CR43                  #
# eye02.jpg: T90, CR43                  #
# cyclist03.jpg (wheel): T215, CR56     # 
# cyclist03.jpg (sprocket): T93, CR11   #
#########################################

def main():
    sg.theme("Purple")

    # Define the window layout
    layout = [
        [sg.Text("Hough Circle Transform", size=(60, 1), justification="center")],
        [sg.Image(filename="", key="-IMAGE-")],
        [sg.Text('Load Image')],
        [sg.Input(key='-FILE-', visible=False, enable_events=True), sg.FileBrowse()],
        [
            sg.Text('Threshold:', size=(15, 0)),
            sg.Slider((0, 255), 87, 1, orientation="h", size=(40, 15), key="-THRESH SLIDER-",),
        ],
        [
            sg.Text('Circle Radius:', size=(15, 0)),
            sg.Slider((0, 255), 43, 1, orientation="h", size=(40, 15), key="-CIRCLE RADIUS-",),
        ],
        [sg.Button("Exit", size=(10, 1))],
    ]

    # Create the window, append the layout
    window = sg.Window("Circle Detection", layout, location=(1000, 400))
    
    # Store the initial image in "img" variable
    img = cv2.imread("eye01.jpg")
    
    # Initialize the Hough Circle parameters. 
    # p1 = the higher threshold of the two passed to the Canny edge detector (the lower one is twice smaller)
    # r1 = minimum circle radius
    # r2 = maximum circle radius
    p1 = 90
    p2 = p1 * 0.4
    r1 = 43
    r2 = r1 + 25
    
    while True:
        event, values = window.read(timeout=20)
        
        if event == "Exit" or event == sg.WIN_CLOSED:
            break
        
        # Trigger when Browse button is pressed to load image
        # Image file directory is stored in "filename" and stored in "img" for further processing
        if event == '-FILE-':
            filename = values['-FILE-']
            window['-FILE-'].update(filename)
            img = cv2.imread(filename)
        
        # Store threshold slider value in p1 and p2 (p1*0.4) whenever slider is adjusted
        if values["-THRESH SLIDER-"]:
            p1 = values["-THRESH SLIDER-"]
            p2 = p1 * 0.4

        # Store circle radius slider value in r1 and r2 (r1+25) whenever slider is adjusted 
        if values["-CIRCLE RADIUS-"]:
            r1 = int(values["-CIRCLE RADIUS-"])
            r2 = r1 + 25
        
        # Calculate scale percent to fix image height at 408, resize image, and copy
        scale_percent = 408/img.shape[0]
        width = int(img.shape[1] * scale_percent)
        height = int(img.shape[0] * scale_percent)
        dsize = (width, height)
        img = cv2.resize(img, dsize, interpolation = cv2.INTER_AREA)
        cimg = np.copy(img)
        
        # Convert from BGR to gray-scale       
        gray = cv2.cvtColor(cimg, cv2.COLOR_BGR2GRAY)
        
        # Detect circles using HoughCircles transform and store position in circles array
        circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1, cimg.shape[0]/64, 
                                   param1=p1, param2=p2, 
                                   minRadius=r1, maxRadius=r2)
        
        # Edge image using Canny edge detector for debugging
        canny = cv2.Canny(gray, p1, p2)
        canny_disp = cv2.cvtColor(canny, cv2.COLOR_GRAY2BGR)
        
     
        # If at least 1 circle is detected
        if circles is not None:
            cir_len = circles.shape[1] # store length of circles found
            circles = np.uint16(np.around(circles))
            for i in circles[0, :]:
                # Coloured image - Draw the outer circle
                cv2.circle(cimg, (i[0], i[1]), i[2], (0, 255, 0), 2)
                # Coloured image - Draw the center of the circle
                cv2.circle(cimg, (i[0], i[1]), 2, (0, 0, 255), 3)
                
                # Canny Edge - Draw the outer circle
                cv2.circle(canny_disp,(i[0],i[1]),i[2],(0,255,0),2)  #img,center,radius,color[, thickness[, lineType[, shift]]]
                # Canny Edge - Draw the center of the circle
                cv2.circle(canny_disp,(i[0],i[1]),2,(0,255,0),3)

        else:
            cir_len = 0 # no circles detected    
        
        # Display image
        img_concat = np.hstack((cimg, canny_disp))
        imgbytes = cv2.imencode(".png", img_concat)[1].tobytes()
        window["-IMAGE-"].update(data=imgbytes)


    window.close()

main()

AttributeError: 'NoneType' object has no attribute 'shape'